# Definición, tipologías y casos de uso de Graph Neural Networks para el aprendizaje basado en relaciones: presentación de implementación

## Introducción al notebook

Este documento de tipo notebook acompaña a la memoria del TFG con título *Definición, tipologías y casos de uso de Graph Neural Networks para el aprendizaje basado en relaciones*. El objetivo del documento es presentar la implementación de las pruebas realizadas con diferentes modelos de redes neuronales gráficas explicadas en el proyecto, así como plantear distintas aproximaciones para su codificación.

Aunque van a explicarse algunos conceptos relacionados con las librerías de Python utilizadas, se deja en manos de la memoria de proyecto el explicar la base teórica de los mismos, así como la evaluación e interpretación de los resultados obtenidos.

## Estructura del documento

Mediante el uso del orden presentado en la memoria, se van a repasar distintas aproximaciones a las GNN con dos planteamientos principales:

1. Clasificación de nodos dentro de un grafo:
    1. Implementación mediante un modelo con base espectral.
    1. Implementación con un modelo espacial mediante paso de mensajes.
    1. Contraste de resultados frente a un modelo tradicional que no tiene en cuenta la estructura de los datos, en busca de justificar la motivación de la existencia de las redes gráficas.
1. Predicción de enlazado.
    1. Implementación de un modelo que hace uso de paso de mensajes para predecir enlaces entre nodos.

Para cada uno de estos ejemplos se presentarán resultados de métricas de precisión durante el proceso de entrenamiento, para lo que se hará uso de datos de validación, así como pruebas finales sobre datos de test.

## Preparación del entorno

### Instalación de módulos

Como primer paso, se procede a instalar los módulos necesarios:

* [numpy](https://numpy.org/), para poder realizar las operaciones necesarias sobre los datos.
* [TensorFlow](https://www.tensorflow.org/), como librería principal para ejecutar los algoritmos de aprendizaje computacional del bloque de clasificación de nodos.
* [Spektral](https://graphneural.network/), una cómoda librería y colección de sets de datos que facilita el trabajo con redes neuronales gráficas.
* [PyTorch](https://pytorch.org/), librería de aprendizaje computacional que será usada para las pruebas de predicción de enlaces. Junto a ella se instalarán también algunas sublibrerías que serán necesarias.
* [tqdm](https://github.com/tqdm/tqdm), una librería simple para crear barras de progreso, con el simple objetivo de facilitar la lectura de algunas pruebas.

A continuación, se hacen las llamadas oportunas para instalar las mismas en el entorno actual. De manera adicional, se adjunta el fichero `requirements.txt` junto con este notebook para su uso en caso de necesidad.

In [2]:
# Install required modules
necessary_modules = ['numpy', 'tensorflow', 'spektral', 'torch', 'torch-cluster', 'torch-scatter', 'torch-sparse', 'torch-geometric', 'tqdm']
for module in necessary_modules:
    !pip install {module}

# Import numpy module, that is going to be use across the project
import numpy as np

### Configuración de TensorBoard

Tanto en los ejemplos donde se hace uso de TensorFlow como en aquellos implementados con PyTorch, se extraerán las métricas de aprendizaje y precisión a ficheros de registro que podrán ser leídos mediante [TensorBoard](https://www.tensorflow.org/tensorboard?hl=en), un entorno del propio TensorFlow que facilita su seguimiento y visualización. 

Para ello, es preciso preparar un directorio donde guardar el historial de registro de mensajes y configurar el propio entorno.

In [7]:
# Import TensorBoard callback to be able to use it in all the code samples
from keras.callbacks import TensorBoard

# Prepare placement for the logs
import os
root_logdir = os.path.join(os.curdir, 'my_logs')

# Create directory if it doesn't exists
from pathlib import Path
Path(root_logdir).mkdir(parents=True, exist_ok=True)

# Define a function that returns the path to store the log depending on the experiment
def get_run_logdir(experiment_name):
    import time
    run_id = time.strftime(f'run_{experiment_name}_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id) 

## Implementación de pruebas

A continuación se presentan las distintas pruebas realizadas.

### Pruebas de clasificación de nodos

Tal como se expone en la memoria de proyecto, las redes neuronales gráficas permiten, entre otras cosas, la clasificación de nodos en base a la información de sus vecindarios, lo que permite asignar clases dentro del contexto de las entidades de un grafo.

Para la realización de estas pruebas se hará uso de la librería Spektral, que ya ha sido instalada con anterioridad. Spektral ofrece una amplia colección de modelos de red neuronal gráfica ya implementados, así como distintas utilidades de código para su explotación. A modo de añadido, también incorpora una clase especial para poder cargar juegos de datos con distintas características, entre los que se encuentra CORA, que ha sido escogido para la realización de estas pruebas.

Así, el primer paso será importar el juego de datos y repasarlo, para lo que será preciso cargar la propia librería Spektral, así como TensorFlow y Keras, que pasan a sumarse a numpy, ya cargado, a la lista de módulos utilizados.

In [10]:
# Import necessary librearies and define aliases to refer to them easier
import spektral
import tensorflow as tf
from tensorflow import keras

#### Carga y preparación del conjunto de datos

Como se indica en la memoria de proyecto, se hará uso del juego de datos conocido como CORA. Gracias a Spektral, es sencillo descargar este juego de datos y sus distintas estructuras ya preparadas.

In [11]:
# Download CORA dataset and its different members
dataset = spektral.datasets.citation.Citation(
    'cora', 
    random_split=False, # split randomly: 20 nodes per class for training, 30 nodes 
        # per class for validation; or "Planetoid" (Yang et al. 2016)
    normalize_x=False,  # normalize the features
    dtype=np.float32 # numpy data type for the graph data
    )

# Also load a list of labels as names, justo to be able to use it
label_names = ['Case_Based', 'Genetic_Algorithms', 'Neural_Networks', 
    'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning', 'Theory']

/Users/ghostmou/virtualenvs/uoc-tfg-gnn/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Tras esto, se pueden consultar las características del conjunto de datos que ha sido descargado.

In [20]:
print(f'Dataset: {dataset}')
print('First graph: {}'.format(dataset.graphs[0]))
print('Node labels: {}'.format(', '.join(label_names)))
print(f'Training samples: {np.sum(dataset.mask_tr)}')
print(f'Validation samples: {np.sum(dataset.mask_va)}')
print(f'Test samples: {np.sum(dataset.mask_te)}')

Dataset: Citation(n_graphs=1)
First graph: Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)
Node labels: Case_Based, Genetic_Algorithms, Neural_Networks, Probabilistic_Methods, Reinforcement_Learning, Rule_Learning, Theory
Training samples: 140
Validation samples: 500
Test samples: 1000


Gracias a los datos extraídos, se puede ver que la variable `dataset` contiene un grafo, el cuál puede ser accedido para conocer sus características. De este modo, se comprueba que el conjunto de datos CORA ha sido descargado y está compuesto por:

* 2708 nodos o vértices, que conforman el grafo.
* 1433 atributos de nodo.
* 0 atributos de relación, es decir, los enlaces que relacionan los nodos no contienen información. Como se explica en la memoria de proyecto, esto impide tener distintas categorías en los vínculos y limitará el aprendizaje al contenido de los nodos y el contexto con el que se relacionan.
* 7 clases. En base a la definición del juego de datos, se sabe que los vértices se clasifican en dicho número de grupos. Sin embargo, como se explica en la memoria de proyecto, también es posible hacer uso de GNNs para clasificar grafos, por lo que Spektral nos permite trabajar con dos tipos de etiquetas: de nodo y de grafo.

Además, Spektral se encarga de generar los subconjuntos del juego de datos que son necesarios para las pruebas a realizar, que quedan divididos en tres:

* Conjunto de entrenamiento, que cuenta con 140 nodos. Es accesible desde `dataset.mask_tr`.
* Conjunto de validación, compuesto por 500 vértices. Puede recuperarse mediante `dataset.mask_va`.
* Conjunto de pruebas o *test*, formado por 1000 entidades. Se pueden computar desde `dataset.mask_te`.

Esta separación en conjuntos de entrenamiento, validación y test se hace mediante un enmascaramiento: están formados por un vector de tipo `narray` con tantas posiciones como nodos hay en el conjunto de datos. En cada posición se registrará un `1` si el vértice correspondiente pertenece al conjunto y con un `0` en caso contrario.

Por esta razón, se presenta la posibilidad de generar los pesos de cada vértice según si han sido seleccionados o no. Con el fin de fijar el mismo peso para todos dentro de cada conjunto, se procede a dividir cada uno de ellos por el número de observaciones en su grupo. El resultado es almacenado en `weighted_mask`.

In [26]:
weighted_mask = [
    mask.astype(np.float32) / np.count_nonzero(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
]

print(f'Training samples weights: {np.nanmean(np.where(weighted_mask[0] > 0, weighted_mask[0],np.nan), 0)}')
print(f'Validation samples weights: {np.nanmean(np.where(weighted_mask[1] > 0, weighted_mask[1],np.nan), 0)}')
print(f'Test samples weights: {np.nanmean(np.where(weighted_mask[2] > 0, weighted_mask[2],np.nan), 0)}')

Training samples weights: 0.0071428571827709675
Validation samples weights: 0.0020000000949949026
Test samples weights: 0.0010000001639127731


Dentro del grafo registrado, es posible acceder a las diferentes estructuras que lo representan

In [36]:
print('Características de los nodos, en formato matriz:\n{}\n\n'.format(dataset.graphs[0].x))
print('Matriz de adyacencia, de tipo sparse matrix:\n{}\n\n'.format(dataset.graphs[0].a))
print('Características de las aristas, en formato matriz (vacía, en este caso):\n{}\n\n'.format(dataset.graphs[0].e))
print('Etiquetas de cada nodo:\n{}'.format(dataset.graphs[0].y))

Características de los nodos, en formato matriz:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Matriz de adyacencia, de tipo sparse matrix:
  (0, 633)	1.0
  (0, 1862)	1.0
  (0, 2582)	1.0
  (1, 2)	1.0
  (1, 652)	1.0
  (1, 654)	1.0
  (2, 1)	1.0
  (2, 332)	1.0
  (2, 1454)	1.0
  (2, 1666)	1.0
  (2, 1986)	1.0
  (3, 2544)	1.0
  (4, 1016)	1.0
  (4, 1256)	1.0
  (4, 1761)	1.0
  (4, 2175)	1.0
  (4, 2176)	1.0
  (5, 1629)	1.0
  (5, 1659)	1.0
  (5, 2546)	1.0
  (6, 373)	1.0
  (6, 1042)	1.0
  (6, 1416)	1.0
  (6, 1602)	1.0
  (7, 208)	1.0
  :	:
  (2694, 431)	1.0
  (2694, 2695)	1.0
  (2695, 431)	1.0
  (2695, 2694)	1.0
  (2696, 2615)	1.0
  (2697, 986)	1.0
  (2698, 1400)	1.0
  (2698, 1573)	1.0
  (2699, 2630)	1.0
  (2700, 1151)	1.0
  (2701, 44)	1.0
  (2701, 2624)	1.0
  (2702, 186)	1.0
  (2702, 1536)	1.0
  (2703, 1298)	1.0
  (2704, 641)	1.0
  (2705, 287)	1.0
  (2706, 165)	1.0
  (2706, 169)	1.0
  (

#### Implementación ConvGNN espectral

Gracias al fichero de datos preparado, es posible probar la implementación de uno de los modelos presentados en la memoria de proyecto: una red neuronal convolucional gráfica o ConvGNN con una aproximación espectral.

Como se explica en el documento de proyecto, el uso de la estructural espectral permite el aprendizaje y predicción en base a la estructura general del grafo, por lo que se hará uso de la matriz de adyacencia y las características de los nodos, pero sin la posibilidad de tener en cuenta la estructura general, algo que veremos en mayor detalle en la implementación de una ConvGNN espacial en el siguiente apartado.

In [37]:
# TODO Cargar los datos a los loaders
# TODO Implementar GCN básica
# TODO Probar con otros modelos mediante GridSearch (si quiero, sino... no)
# TODO Presentar resultados de performance